<a href="https://colab.research.google.com/github/nourhanekefsi/RISCV_singleCycle/blob/master/Projet_PMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!python --version
!nvcc --version

# Installer l'extension CUDA pour Jupyter
!pip install nvcc4jupyter
%load_ext nvcc4jupyter


Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp_ewesjj_".


In [9]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "/content/stb_image.h"
#include "/content/stb_image_write.h"

__global__ void thresholding(unsigned char *image, int seuil, int n, unsigned char *imageResult) {
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    if (idx < n) {
        imageResult[idx] = (image[idx] < seuil) ? 0 : 255;
    }
}

int main() {
    clock_t start_total, end_total;
    start_total = clock();

    int width, height, channels;

    unsigned char *image = stbi_load("/content/image-1.jpg", &width, &height, &channels, 1);

    if (image == NULL) {
        printf("Erreur lors du chargement de l'image\n");
        return 1;
    }

    int n = width * height;

    unsigned char *h_imageResult = (unsigned char*)malloc(n * sizeof(unsigned char));

    unsigned char *d_image, *d_imageResult;
    cudaMalloc((void**)&d_image, n * sizeof(unsigned char));
    cudaMalloc((void**)&d_imageResult, n * sizeof(unsigned char));

    cudaMemcpy(d_image, image, n * sizeof(unsigned char), cudaMemcpyHostToDevice);

    int seuil = 128;

    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;

    thresholding<<<numBlocks, blockSize>>>(d_image, seuil, n, d_imageResult);

    cudaDeviceSynchronize();

    cudaMemcpy(h_imageResult, d_imageResult, n * sizeof(unsigned char), cudaMemcpyDeviceToHost);
    end_total = clock();
    double total_time = ((double)(end_total - start_total)) / CLOCKS_PER_SEC;
    printf("Temps total d'exécution : %.3f secondes\n", total_time);

    stbi_write_png("thresholded_image.png", width, height, 1, h_imageResult, width);

    stbi_image_free(image);
    free(h_imageResult);
    cudaFree(d_image);
    cudaFree(d_imageResult);

    return 0;
}


Temps total d'exécution : 0.204 secondes

